In [446]:
# Guide for Tuturial
#https://github.com/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/official/prediction/custom_batch_prediction_feature_filter.ipynb
#https://cloud.google.com/vertex-ai/docs/tutorials/train-tensorflow-bigquery
#https://cloud.google.com/vertex-ai/docs/tutorials/tabular-bq-prediction

In [447]:
import argparse
import os
from typing import Tuple, Optional

import pandas as pd
import numpy as np
import tensorflow as tf

from google.cloud import bigquery
from google.cloud import storage

from sklearn.model_selection import train_test_split

In [448]:
EPOCHS = 100
BATCH_SIZE = 32
TRAIN_STRATEGY = "single"


mean_and_std_json_file="incident_mean_and_std.json"

LABEL_COLUMN = "severity_name"
UNUSED_COLUMNS = ['severity_id']

model_dir='model'

In [449]:
def load_data_bq(sql:str):
 client_bq = bigquery.Client()
 query_result=client_bq.query(sql)
 df=query_result.to_dataframe()
 return df

def download_table(bq_table_uri: str):
    # Remove bq:// prefix if present
    bqclient= bigquery.Client()
    prefix = "bq://"
    if bq_table_uri.startswith(prefix):
        bq_table_uri = bq_table_uri[len(prefix) :]

    table = bigquery.TableReference.from_string(bq_table_uri)
    rows = bqclient.list_rows(
        table,
    )
    return rows.to_dataframe()
df = download_table("pongthorn.SMartML.TrainEval_Incident_20230316")

sr_predict=df.iloc[-1,:]
df=df.iloc[0:len(df)-1,:]

print(df.info())
df.tail(3)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2142 entries, 0 to 2141
Data columns (total 9 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   severity_id                2142 non-null   Int64  
 1   severity_name              2142 non-null   object 
 2   sla                        2142 non-null   object 
 3   product_type               2142 non-null   object 
 4   brand                      2142 non-null   object 
 5   service_type               2142 non-null   object 
 6   incident_type              2142 non-null   object 
 7   open_to_close_hour         2142 non-null   float64
 8   response_to_resolved_hour  2142 non-null   float64
dtypes: Int64(1), float64(2), object(6)
memory usage: 152.8+ KB
None


,severity_id,severity_name,sla,product_type,brand,service_type,incident_type,open_to_close_hour,response_to_resolved_hour
2139,4,Cosmatic,24x7 4Hrs Response Time,Software,Trend Micro,Request,General Incident,168.150000,164.666667
2140,4,Cosmatic,24x7 4Hrs Response Time,Software,Trend Micro,Request,General Incident,163.850000,163.850000
2141,4,Cosmatic,24x7 6Hrs Resolution Time,Software,Trend Micro,Request,General Incident,5.483333,5.483333


In [450]:
list_label=df[LABEL_COLUMN].unique()
print(list_label)

cate_sla=df['sla'].unique()
print(cate_sla)

cate_productType=df['product_type'].unique()
print(cate_productType)

cate_brand=df['brand'].unique()
print(cate_brand)

cate_serviceType=df['service_type'].unique()
print(cate_serviceType)

cate_incidentType=df['incident_type'].unique()
print(cate_incidentType)


_CATEGORICAL_TYPES = {  
    LABEL_COLUMN:pd.api.types.CategoricalDtype(categories=list_label),
    "sla": pd.api.types.CategoricalDtype(categories=cate_sla),
    "product_type": pd.api.types.CategoricalDtype(categories=cate_productType),
    "brand": pd.api.types.CategoricalDtype(categories=cate_brand),
    "service_type": pd.api.types.CategoricalDtype(categories=cate_serviceType),
    "incident_type": pd.api.types.CategoricalDtype(categories=cate_incidentType),
}
# print(_CATEGORICAL_TYPES)

['Major' 'Minor' 'Cosmatic' 'Critical']
['24x7 4Hrs Resolution Time' '8x5 4Hrs Response Time'
 '24x7 4Hrs Response Time' '9x5xNBD 4Hrs Response Time'
 '24x7 6Hrs Response Time' '24x7 6Hrs Resolution Time']
['Other' 'Storage' 'Software' 'Server' 'Hardware' 'Switch' 'Access Point'
 'Security' 'Tape Library' 'Firewall']
['F5' 'EMC' 'HPE' 'IBM' 'YIP' 'DELL' 'Cisco' 'SAPB1' 'Veeam' 'NetApp'
 'Oracle' 'VMWare' 'eUnite' 'Nutanix' 'Red Hat' 'Veritas' 'Broadcom'
 'Fortinet' 'Microsoft' 'Palo Alto' 'CheckPoint' 'Trend Micro']
['Request' 'Incident']
['General Incident' 'Software' 'Maintenance System'
 'Hard Disk Drive Failure' 'Controller/Node Failure'
 'Network Adapter Failure' 'Report' 'Network Card Failure'
 'Memory Failure' 'System Board Failure' 'CPU Failure'
 'Power Supply Failure' 'Battery Failure' 'Other Failure'
 'Cache Battery Failure' 'OS / Firmware' 'Configuration Change'
 'Upgrade Software' 'Fan Failure' 'Network Cable Failure' 'Backup Failure']


In [451]:
def download_mean_and_std(mean_and_std_json_file):
    """Download mean and std for each column"""
    import json
    file_path=mean_and_std_json_file
    # bucket, file_path = extract_bucket_and_prefix_from_gcs_path(mean_and_std_json_file)
    # download_blob(bucket_name=bucket, source_blob_name=file_path, destination_file_name=file_path)
    
    with open(file_path, 'r') as file:
        return json.loads(file.read())

In [452]:
def preprocess(df):
    """Converts categorical features to numeric. Removes unused columns.

    Args:
      df: Pandas df with raw data

    Returns:
      df with preprocessed data
    """
    df = df.drop(columns=UNUSED_COLUMNS)

    # Drop rows with NaN's
    df = df.dropna()

    # Convert integer valued (numeric) columns to floating point
    numeric_columns = df.select_dtypes(["int32", "float32", "float64"]).columns
    df[numeric_columns] = df[numeric_columns].astype("float32")

    # Convert categorical columns to numeric
    cat_columns = df.select_dtypes(["object"]).columns

    df[cat_columns] = df[cat_columns].apply(
        lambda x: x.astype(_CATEGORICAL_TYPES[x.name])
    )
    df[cat_columns] = df[cat_columns].apply(lambda x: x.cat.codes)
    return df

In [453]:
def standardize(df, mean_and_std):
    """Scales numerical columns using their means and standard deviation to get
    z-scores: the mean of each numerical column becomes 0, and the standard
    deviation becomes 1. This can help the model converge during training.

    Args:
      df: Pandas df

    Returns:
      Input df with the numerical columns scaled to z-scores
    """
    dtypes = list(zip(df.dtypes.index, map(str, df.dtypes)))
    # Normalize numeric columns.
    for column, dtype in dtypes:
        if dtype == "float32":
            df[column] -= mean_and_std[column]["mean"]
            df[column] /= mean_and_std[column]["std"]
    return df


In [454]:
def convert_dataframe_to_dataset(
    df_train,
    df_validation,
    mean_and_std
):

    df_train_x, df_train_y = df_train, df_train.pop(LABEL_COLUMN)
    df_validation_x, df_validation_y = df_validation, df_validation.pop(LABEL_COLUMN)

    # Join train_x and eval_x to normalize on overall means and standard
    # deviations. Then separate them again.
    all_x = pd.concat([df_train_x, df_validation_x], keys=["train", "eval"])
    all_x = standardize(all_x, mean_and_std)
    df_train_x, df_validation_x = all_x.xs("train"), all_x.xs("eval")

    y_train = np.asarray(df_train_y).astype("float32")
    y_validation = np.asarray(df_validation_y).astype("float32")

    # Convert to numpy representation
    x_train = np.asarray(df_train_x)
    x_test = np.asarray(df_validation_x)

    # Convert to one-hot representation
    y_train = tf.keras.utils.to_categorical(y_train, num_classes=len(list_label))
    y_validation = tf.keras.utils.to_categorical(y_validation, num_classes=len(list_label))
    
    print(x_train.shape,y_train.shape, x_test.shape,y_validation.shape)
    
    # return   x_train,y_train, x_test,y_validation

    dataset_train = tf.data.Dataset.from_tensor_slices((x_train, y_train))
    dataset_validation = tf.data.Dataset.from_tensor_slices((x_test, y_validation))
    
    return (dataset_train, dataset_validation)

In [455]:
def create_model(num_features,num_class):
    # Create model
    Dense = tf.keras.layers.Dense
    model = tf.keras.Sequential(
        [
            Dense(
                16,
                activation=tf.nn.relu,
                input_dim=num_features,
            ),
            Dense(16, activation=tf.nn.relu),
            # Dense(50, activation=tf.nn.relu),
            # Dense(25, activation=tf.nn.relu),
            Dense(num_class, activation=tf.nn.softmax),
        ]
    )
    
    # Compile Keras model
    # optimizer = tf.keras.optimizers.RMSprop(lr=0.001)
    model.compile(
        loss="categorical_crossentropy", metrics=["accuracy"], optimizer='adam'
    )
    
    return model

In [456]:
mean_and_std = download_mean_and_std(mean_and_std_json_file)
print(mean_and_std)

{'open_to_close_hour': {'mean': 95.90196856520386, 'std': 262.5754868829863}, 'response_to_resolved_hour': {'mean': 70.41082321817616, 'std': 217.11239323048156}}


In [457]:
len_all=len(df)
print(f"All Data = {len_all}")

df_train,df_validation = train_test_split(df,test_size=0.2)
# df_validation,df_test = train_test_split(df_validation, test_size=0.5)

len_all=len(df)
len_train=len(df_train)
len_val=len(df_validation)
# len_test=len(df_test)

print(f'{len_train} =train examples ({round(len_train/len_all*100,1)}%)')
print(f'{len_val} =val examples ({round(len_val/len_all*100,1)}%)')
# print(f'{len_test} =test examples ({round(len_test/len_all*100,1)}%)')

All Data = 2142
1713 =train examples (80.0%)
429 =val examples (20.0%)


In [458]:
df_train = preprocess(df_train)
df_validation = preprocess(df_validation)
# df_test = preprocess(df_test)
print(df_train.info())
df_train.head()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1713 entries, 1532 to 1886
Data columns (total 8 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   severity_name              1713 non-null   int8   
 1   sla                        1713 non-null   int8   
 2   product_type               1713 non-null   int8   
 3   brand                      1713 non-null   int8   
 4   service_type               1713 non-null   int8   
 5   incident_type              1713 non-null   int8   
 6   open_to_close_hour         1713 non-null   float32
 7   response_to_resolved_hour  1713 non-null   float32
dtypes: float32(2), int8(6)
memory usage: 36.8 KB
None


,severity_name,sla,product_type,brand,service_type,incident_type,open_to_close_hour,response_to_resolved_hour
1532,1,0,4,10,1,3,6.000000,2.000000
1382,0,0,3,10,1,3,49.983334,47.000000
133,0,0,3,2,0,0,6.650000,6.400000
1945,2,2,2,11,0,0,8.466666,8.000000
914,0,0,5,6,0,0,5.133333,4.133333


In [459]:
dataset_train, dataset_validation = convert_dataframe_to_dataset(
  df_train, 
  df_validation, 
  mean_and_std
)
dataset_train = dataset_train.shuffle(len(df_train))


(1713, 7) (1713, 4) (429, 7) (429, 4)


In [460]:
model = create_model( num_features=dataset_train._flat_shapes[0].dims[0].value,num_class=len(list_label))
model.summary()

Model: "sequential_10"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_30 (Dense)            (None, 16)                128       
                                                                 
 dense_31 (Dense)            (None, 16)                272       
                                                                 
 dense_32 (Dense)            (None, 4)                 68        
                                                                 
Total params: 468
Trainable params: 468
Non-trainable params: 0
_________________________________________________________________


In [461]:
dataset_train = dataset_train.batch(BATCH_SIZE)
dataset_validation = dataset_validation.batch(BATCH_SIZE)

In [462]:
history=model.fit(dataset_train, epochs=EPOCHS, validation_data=dataset_validation,batch_size=BATCH_SIZE)

Epoch 1/100
54/54 [==============================] - 1s 5ms/step - loss: 1.8150 - accuracy: 0.2405 - val_loss: 1.4280 - val_accuracy: 0.4382
Epoch 2/100
54/54 [==============================] - 0s 3ms/step - loss: 1.3098 - accuracy: 0.4717 - val_loss: 1.2250 - val_accuracy: 0.4802
Epoch 3/100
54/54 [==============================] - 0s 3ms/step - loss: 1.1664 - accuracy: 0.4869 - val_loss: 1.1259 - val_accuracy: 0.4802
Epoch 4/100
54/54 [==============================] - 0s 3ms/step - loss: 1.1006 - accuracy: 0.5038 - val_loss: 1.0930 - val_accuracy: 0.5291
Epoch 5/100
54/54 [==============================] - 0s 3ms/step - loss: 1.0765 - accuracy: 0.5301 - val_loss: 1.0791 - val_accuracy: 0.5175
Epoch 6/100
54/54 [==============================] - 0s 3ms/step - loss: 1.0613 - accuracy: 0.5447 - val_loss: 1.0650 - val_accuracy: 0.5548
Epoch 7/100
54/54 [==============================] - 0s 3ms/step - loss: 1.0461 - accuracy: 0.5593 - val_loss: 1.0537 - val_accuracy: 0.5408
Epoch 8/100
5

In [463]:
tf.saved_model.save(model, model_dir)

INFO:tensorflow:Assets written to: model\assets


INFO:tensorflow:Assets written to: model\assets
